# Chapter 18: Machine Learning from Experiment - Counterfactual Learning

This notebook contains all code examples from Chapter 18, demonstrating counterfactual learning methods for training ML models from experiment data.

## Setup: Install Required Packages

In [ ]:
# Install required packages (uncomment if needed)
# !pip install pandas numpy scikit-learn econml

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.utils import resample
from sklearn.isotonic import IsotonicRegression
import warnings
warnings.filterwarnings('ignore')

## Section 2.2.4: Instrumental Variables (IV)

Using clean control data to correct contaminated production data.

In [13]:
# Note: This example requires econml library
# The IV method uses control data to debias production data

try:
    from econml.iv.dml import DMLIV
    
    def train_with_iv_correction(control_data, production_data):
        """
        Use clean control data to correct contaminated production data.
        
        Args:
            control_data: Small dataset with random assignments (unbiased)
            production_data: Large dataset with model-influenced assignments (biased)
        """
        # Create instrument: 1 for random assignment, 0 for model assignment
        control_data['instrument'] = 1
        production_data['instrument'] = 0
        combined_data = pd.concat([control_data, production_data])
        
        X = combined_data[['user_age', 'song_genre', 'time_of_day']]
        Y = combined_data['engagement']
        T = combined_data['song_shown']  # Treatment (which song shown)
        Z = combined_data['instrument']  # Instrument (random vs model)
        
        # Train IV model using control data to debias production data
        iv_model = DMLIV(
            model_y_xw=RandomForestRegressor(),
            model_t_xw=RandomForestRegressor(),
            model_t_xwz=RandomForestRegressor(),
            discrete_treatment=False
        )
        
        iv_model.fit(Y, T, X=X, Z=Z)
        return iv_model
    
    print("✓ IV function defined successfully")
    print("Note: Requires control_data and production_data to run")
    
except ImportError:
    print("⚠ econml not installed. Install with: pip install econml")
    print("This is an advanced method - skipping for basic demo")

⚠ econml not installed. Install with: pip install econml
This is an advanced method - skipping for basic demo


## Section 2.3: Use Case 1 - Personalized Recommendations

### Step 1: Collect Experiment Data

In [14]:
# Experiment data: (user_features, treatment, CTR)
experiment_data = pd.DataFrame({
    'user_id': [1, 2, 3, 4, 5, 6],
    'age': [25, 34, 45, 29, 52, 38],
    'num_past_purchases': [2, 10, 0, 5, 15, 3],
    'treatment': [1, 0, 1, 0, 1, 0],  # 1=personalized, 0=popular
    'ctr': [0.08, 0.05, 0.03, 0.06, 0.12, 0.04]
})

print("Experiment Data:")
print(experiment_data)
print(f"\nTreatment group size: {(experiment_data['treatment'] == 1).sum()}")
print(f"Control group size: {(experiment_data['treatment'] == 0).sum()}")

Experiment Data:
   user_id  age  num_past_purchases  treatment   ctr
0        1   25                   2          1  0.08
1        2   34                  10          0  0.05
2        3   45                   0          1  0.03
3        4   29                   5          0  0.06
4        5   52                  15          1  0.12
5        6   38                   3          0  0.04

Treatment group size: 3
Control group size: 3


### Step 2: Train a Counterfactual Model (T-Learner)

In [4]:
# Separate treatment and control data
treatment_data = experiment_data[experiment_data['treatment'] == 1]
control_data = experiment_data[experiment_data['treatment'] == 0]

X_features = ['age', 'num_past_purchases']

# Train two models
model_treatment = RandomForestRegressor(n_estimators=100, random_state=42)
model_treatment.fit(treatment_data[X_features], treatment_data['ctr'])

model_control = RandomForestRegressor(n_estimators=100, random_state=42)
model_control.fit(control_data[X_features], control_data['ctr'])

print("✓ T-Learner models trained successfully")
print(f"  Treatment model: {len(treatment_data)} samples")
print(f"  Control model: {len(control_data)} samples")

✓ T-Learner models trained successfully
  Treatment model: 3 samples
  Control model: 3 samples


### Step 3: Predict Individual Treatment Effects

In [5]:
# For a new user, predict CTR under both scenarios
new_user = pd.DataFrame({'age': [30], 'num_past_purchases': [7]})

ctr_personalized = model_treatment.predict(new_user)[0]
ctr_popular = model_control.predict(new_user)[0]

uplift = ctr_personalized - ctr_popular
print(f"Predicted CTR (personalized): {ctr_personalized:.4f}")
print(f"Predicted CTR (popular): {ctr_popular:.4f}")
print(f"Expected uplift: {uplift:.4f}")

# Decision rule: Show personalized recommendations if uplift > threshold
if uplift > 0.01:
    print("\n✓ Recommend: Personalized")
else:
    print("\n✓ Recommend: Popular")

Predicted CTR (personalized): 0.0696
Predicted CTR (popular): 0.0563
Expected uplift: 0.0133

✓ Recommend: Personalized


### Step 3b: Adding Uncertainty Estimates

#### Method 1: Using Random Forest's Built-in Uncertainty

In [6]:
def predict_with_uncertainty_rf(model, X):
    """
    Use Random Forest's ensemble of trees to estimate uncertainty.
    
    Returns:
        mean: Point prediction (average across trees)
        std: Standard error (variance across trees)
        ci_lower, ci_upper: 95% confidence interval
    """
    # Get predictions from each tree in the forest
    tree_predictions = np.array([tree.predict(X) for tree in model.estimators_])
    
    mean_pred = tree_predictions.mean(axis=0)[0]
    std_pred = tree_predictions.std(axis=0)[0]
    
    # 95% confidence interval
    ci_lower = np.percentile(tree_predictions, 2.5, axis=0)[0]
    ci_upper = np.percentile(tree_predictions, 97.5, axis=0)[0]
    
    return mean_pred, std_pred, ci_lower, ci_upper

print("✓ Random Forest uncertainty function defined")

✓ Random Forest uncertainty function defined


#### Method 2: Bootstrap Resampling (More Robust for Small Data)

In [15]:
def predict_with_bootstrap(model_class, X_train, y_train, X_test, n_bootstrap=100):
    """
    Bootstrap the training data to estimate prediction uncertainty.
    
    Args:
        model_class: Model class to instantiate (e.g., RandomForestRegressor)
        X_train, y_train: Training data
        X_test: Test data for prediction
        n_bootstrap: Number of bootstrap resamples (typically 50-200)
        
    Returns:
        mean, std, ci_lower, ci_upper
    """
    predictions = []
    
    for i in range(n_bootstrap):
        # Resample training data with replacement
        X_boot, y_boot = resample(X_train, y_train, random_state=i)
        
        # Train model on bootstrap sample
        model = model_class(n_estimators=50, random_state=i)
        model.fit(X_boot, y_boot)
        
        # Predict on test data
        pred = model.predict(X_test)[0]
        predictions.append(pred)
    
    predictions = np.array(predictions)
    
    mean_pred = predictions.mean()
    std_pred = predictions.std()
    ci_lower = np.percentile(predictions, 2.5)
    ci_upper = np.percentile(predictions, 97.5)
    
    return mean_pred, std_pred, ci_lower, ci_upper

print("✓ Bootstrap uncertainty function defined")

✓ Bootstrap uncertainty function defined


#### Apply Uncertainty Estimation

In [16]:
# Choose method based on data size
if len(treatment_data) < 1000:
    # Use bootstrap for small datasets (more accurate uncertainty)
    print("Using bootstrap method (n=100 resamples)...")
    ctr_pers_mean, ctr_pers_std, ctr_pers_lower, ctr_pers_upper = \
        predict_with_bootstrap(
            RandomForestRegressor,
            treatment_data[X_features], 
            treatment_data['ctr'],
            new_user,
            n_bootstrap=100
        )
    
    ctr_pop_mean, ctr_pop_std, ctr_pop_lower, ctr_pop_upper = \
        predict_with_bootstrap(
            RandomForestRegressor,
            control_data[X_features],
            control_data['ctr'],
            new_user,
            n_bootstrap=100
        )
else:
    # Use RF built-in for large datasets (faster)
    print("Using Random Forest built-in uncertainty...")
    ctr_pers_mean, ctr_pers_std, ctr_pers_lower, ctr_pers_upper = \
        predict_with_uncertainty_rf(model_treatment, new_user)
    
    ctr_pop_mean, ctr_pop_std, ctr_pop_lower, ctr_pop_upper = \
        predict_with_uncertainty_rf(model_control, new_user)

print("\n✓ Uncertainty estimates computed")

Using bootstrap method (n=100 resamples)...

✓ Uncertainty estimates computed

✓ Uncertainty estimates computed


#### Computing Uplift with Uncertainty

In [9]:
# Uplift uncertainty (using variance propagation)
uplift_mean = ctr_pers_mean - ctr_pop_mean
uplift_std = np.sqrt(ctr_pers_std**2 + ctr_pop_std**2)
uplift_lower = ctr_pers_lower - ctr_pop_upper  # Conservative bound
uplift_upper = ctr_pers_upper - ctr_pop_lower

print(f"\nPredicted CTR (personalized): {ctr_pers_mean:.4f} ± {ctr_pers_std:.4f}")
print(f"  95% CI: [{ctr_pers_lower:.4f}, {ctr_pers_upper:.4f}]")
print(f"\nPredicted CTR (popular): {ctr_pop_mean:.4f} ± {ctr_pop_std:.4f}")
print(f"  95% CI: [{ctr_pop_lower:.4f}, {ctr_pop_upper:.4f}]")
print(f"\nExpected uplift: {uplift_mean:.4f} ± {uplift_std:.4f}")
print(f"  95% CI: [{uplift_lower:.4f}, {uplift_upper:.4f}]")


Predicted CTR (personalized): 0.0695 ± 0.0205
  95% CI: [0.0300, 0.1200]

Predicted CTR (popular): 0.0546 ± 0.0052
  95% CI: [0.0400, 0.0600]

Expected uplift: 0.0150 ± 0.0211
  95% CI: [-0.0300, 0.0800]


#### Risk-Aware Decision Rules

In [10]:
def risk_aware_decision(uplift_mean, uplift_std, uplift_lower, 
                       threshold=0.01, confidence_threshold=0.95):
    """
    Make risk-aware treatment decisions.
    
    Strategies:
    1. Optimistic: Choose if expected uplift > threshold
    2. Conservative: Choose only if lower bound > threshold (high confidence)
    3. Thompson Sampling: Sample from uplift distribution and choose best
    """
    # Strategy 1: Optimistic (use point estimate)
    optimistic_choice = "Personalized" if uplift_mean > threshold else "Popular"
    
    # Strategy 2: Conservative (require high confidence)
    conservative_choice = "Personalized" if uplift_lower > threshold else "Popular"
    
    # Strategy 3: Thompson Sampling (sample from distribution)
    uplift_sample = np.random.normal(uplift_mean, uplift_std)
    thompson_choice = "Personalized" if uplift_sample > threshold else "Popular"
    
    # Strategy 4: Upper Confidence Bound (balance exploration/exploitation)
    ucb_value = uplift_mean + 1.96 * uplift_std  # Optimistic estimate
    ucb_choice = "Personalized" if ucb_value > threshold else "Popular"
    
    return {
        'optimistic': optimistic_choice,
        'conservative': conservative_choice,
        'thompson_sampling': thompson_choice,
        'ucb': ucb_choice,
        'certainty': 1 - (uplift_std / abs(uplift_mean)) if uplift_mean != 0 else 0
    }

decisions = risk_aware_decision(uplift_mean, uplift_std, uplift_lower)

print(f"\n=== Decision Strategies ===")
print(f"Optimistic (point estimate):     {decisions['optimistic']}")
print(f"Conservative (high confidence):  {decisions['conservative']}")
print(f"Thompson Sampling:               {decisions['thompson_sampling']}")
print(f"UCB (exploration bonus):         {decisions['ucb']}")
print(f"Prediction certainty:            {decisions['certainty']:.1%}")


=== Decision Strategies ===
Optimistic (point estimate):     Personalized
Conservative (high confidence):  Popular
Thompson Sampling:               Popular
UCB (exploration bonus):         Personalized
Prediction certainty:            -41.2%


## Section 2.4: Use Case 2 - Multi-Treatment Personalization

Choosing among 3+ treatments using Causal Forests.

In [11]:
# Note: This example requires econml library
try:
    from econml.dml import CausalForestDML
    
    # Create synthetic multi-treatment data for demonstration
    np.random.seed(42)
    n_samples = 300
    
    data = pd.DataFrame({
        'age': np.random.randint(18, 65, n_samples),
        'past_purchases': np.random.randint(0, 20, n_samples),
        'treatment': np.random.choice(['discount', 'free_ship', 'bundle'], n_samples),
        'conversion': np.random.binomial(1, 0.15, n_samples)  # Base 15% conversion
    })
    
    # Train separate causal forests for each treatment vs baseline
    treatments = ['discount', 'free_ship', 'bundle']
    causal_models = {}
    
    for treatment in treatments:
        # Create binary treatment indicator: this treatment vs all others
        T = (data['treatment'] == treatment).astype(int)
        X = data[['age', 'past_purchases']]
        Y = data['conversion']
        
        model = CausalForestDML(
            model_y=RandomForestRegressor(n_estimators=100),
            model_t=RandomForestClassifier(n_estimators=100)
        )
        model.fit(Y, T, X=X, W=None)
        causal_models[treatment] = model
    
    # Predict uplift for each treatment, choose best
    def personalized_promotion(user_features):
        """Assign promotion based on predicted uplift."""
        uplifts = {
            treatment: model.effect(user_features)[0]
            for treatment, model in causal_models.items()
        }
        
        best_treatment = max(uplifts, key=uplifts.get)
        return best_treatment, uplifts
    
    # Example
    user = pd.DataFrame({'age': [25], 'past_purchases': [3]})
    treatment, all_uplifts = personalized_promotion(user)
    print(f"✓ Recommended treatment: {treatment}")
    print(f"  Predicted uplifts: {all_uplifts}")
    
except ImportError:
    print("⚠ econml not installed. Install with: pip install econml")
    print("This demonstrates multi-treatment personalization using Causal Forests")

⚠ econml not installed. Install with: pip install econml
This demonstrates multi-treatment personalization using Causal Forests


## Section 2.5: Use Case 3 - Model Calibration

Correcting systematic biases using experiment data as ground truth.

In [12]:
# Experiment data: predicted CTR vs actual CTR
experiment_results = pd.DataFrame({
    'predicted_ctr': [0.10, 0.08, 0.05, 0.12, 0.15],
    'actual_ctr': [0.08, 0.07, 0.04, 0.10, 0.13],
    'is_new_user': [1, 1, 0, 1, 0]
})

print("Experiment Results (Predicted vs Actual):")
print(experiment_results)

# Train calibrator for new users
new_user_data = experiment_results[experiment_results['is_new_user'] == 1]
calibrator = IsotonicRegression(out_of_bounds='clip')
calibrator.fit(new_user_data['predicted_ctr'], new_user_data['actual_ctr'])

# Apply calibration
def calibrated_ctr_prediction(predicted_ctr, is_new_user):
    if is_new_user:
        return calibrator.predict([predicted_ctr])[0]
    else:
        return predicted_ctr  # No adjustment for existing users

# Test
test_pred = 0.10
calibrated = calibrated_ctr_prediction(test_pred, is_new_user=True)
print(f"\n✓ Calibration complete")
print(f"  Original prediction: {test_pred:.4f}")
print(f"  Calibrated prediction: {calibrated:.4f}")
print(f"  Adjustment: {(calibrated - test_pred):.4f}")

Experiment Results (Predicted vs Actual):
   predicted_ctr  actual_ctr  is_new_user
0           0.10        0.08            1
1           0.08        0.07            1
2           0.05        0.04            0
3           0.12        0.10            1
4           0.15        0.13            0

✓ Calibration complete
  Original prediction: 0.1000
  Calibrated prediction: 0.0800
  Adjustment: -0.0200


## Summary

This notebook demonstrated:

1. **T-Learner for Binary Treatment**: Training separate models for treatment and control groups
2. **Uncertainty Quantification**: Using Random Forest variance and bootstrap methods
3. **Risk-Aware Decisions**: Multiple strategies (optimistic, conservative, Thompson sampling, UCB)
4. **Multi-Treatment Personalization**: Using Causal Forests for 3+ treatments (requires econml)
5. **Model Calibration**: Correcting systematic biases using isotonic regression

All code examples are 100% consistent with Chapter 18 content.